# Housing norm
> Regresión

In [13]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
from functools import reduce

# https://xgboost.readthedocs.io/en/latest/python/python_api.html
import xgboost as xgb

# Cargar mi repo como paquete
from sklearn.datasets import load_boston
import sys
from os.path import abspath

try:
    from src.gaparsimony import GAparsimony
except:
    sys.path.append(abspath("C:/Users/Millan/Desktop/GAparsimony"))
    from src.gaparsimony import GAparsimony

In [14]:
df = pd.read_csv("../../data/housing_norm.csv")
print(df.shape)
df.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,class
0,0.000000,0.18,0.067815,0,0.314815,0.577505,0.641607,0.269203,0.000000,0.208015,0.287234,1.000000,0.089680,0.422222
1,0.000236,0.00,0.242302,0,0.172840,0.547998,0.782698,0.348962,0.043478,0.104962,0.553191,1.000000,0.204470,0.368889
2,0.000236,0.00,0.242302,0,0.172840,0.694386,0.599382,0.348962,0.043478,0.104962,0.553191,0.989737,0.063466,0.660000
3,0.000293,0.00,0.063050,0,0.150206,0.658555,0.441813,0.448545,0.086957,0.066794,0.648936,0.994276,0.033389,0.631111
4,0.000705,0.00,0.063050,0,0.150206,0.687105,0.528321,0.448545,0.086957,0.066794,0.648936,1.000000,0.099338,0.693333


In [15]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

data_train = pd.DataFrame(X_train, columns=df.columns[:-1]).reset_index(drop=True)
data_test = pd.DataFrame(X_test, columns=df.columns[:-1]).reset_index(drop=True)

print(f"train: {data_train.shape} test: {data_test.shape}")

train: (404, 13) test: (102, 13)


In [4]:
data_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.168788,0.00,0.646628,0,0.471193,0.333972,0.972194,0.088307,1.000000,0.914122,0.808511,0.880428,0.639625
1,0.006981,0.00,0.281525,0,0.314815,0.435524,0.552008,0.306359,0.130435,0.229008,0.893617,0.996772,0.185982
2,0.000319,0.35,0.205279,0,0.108848,0.473271,0.210093,0.501150,0.000000,0.223282,0.457447,0.912628,0.168322
3,0.079174,0.00,0.646628,0,0.471193,0.487066,0.846550,0.081132,1.000000,0.914122,0.808511,0.005547,0.594923
4,0.008087,0.00,0.281525,0,0.314815,0.415022,0.685891,0.242514,0.130435,0.229008,0.893617,0.984997,0.263521


In [5]:
data_test.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000961,0.0,0.131598,0,0.257202,0.547040,0.836251,0.137921,0.173913,0.208015,0.425532,0.996470,0.201711
1,0.000563,0.4,0.218109,1,0.127572,0.612569,0.308960,0.268076,0.130435,0.127863,0.531915,1.000000,0.049669
2,0.001117,0.0,1.000000,0,0.460905,0.464074,0.987642,0.067155,0.130435,1.000000,0.797872,0.982879,0.450883
3,0.000959,0.0,0.379399,0,0.057613,0.479785,0.050463,0.378079,0.130435,0.225191,0.702128,0.984896,0.104581
4,0.057141,0.0,0.646628,0,0.674897,0.524238,0.915551,0.112632,1.000000,0.914122,0.808511,0.970220,0.428808


In [21]:
tuneGrid = {
                "n_estimators ": int(586),
                "max_depth": int(13),
                "min_child_weight": int(14),
                "reg_alpha": 2.23530240e-02,
                "reg_lambda": 5.83069253e-02,
                "subsample": 6.28764071e-01,
                "colsample_bytree": 9.74873827e-01,
                "learning_rate": 0.01,
                "random_state": 1234,
                "verbosity": 0}
model = xgb.XGBRegressor(**tuneGrid)
model.fit(data_train.LSTAT[:,np.newaxis], y_train)

rmse_test = mean_squared_error(model.predict(data_test.LSTAT[:, np.newaxis]), y_test)
print(rmse_test)
# LSTAT  

0.016817297836211775
